# <center/>pytorch&class note<center/>

In [ ]:
#https://blog.csdn.net/lee813/article/details/89609691介绍训练的一个blog
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
model = TwoLayerNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
def train():
    for t in range(500):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)
        # Compute and print loss
        loss = criterion(y_pred, y) # 计算损失函数
        print(t, loss.item())
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度
        loss.backward() # loss反向传播
        optimizer.step() # 反向传播后参数更新 

+ 上面一段代码介绍了用`pytorch`训练模型的最基本的使用
+ 在模型编写时候要重写网络结构，编写前向反馈函数
+ 选择损失函数和优化器

# 0511上课笔记
+ 卷积神经网络

前馈神经网络的缺点：数据量太大，连接太多

因此需要卷积

卷积+激活函数（非线性映射）

+ 池化操作：对输入特征进行下采用，获得最主要信息

常用池化操作：最大池化和平均池化

全连接层 分类层

神经网络正则化 为了环节过拟合问题，采取正则化技术提升泛化能力（dropout、batch normalization）

+ 

一般处理序列数据

反向传播

因为tanh的函数导数取值在0-1之间，太多参数乘在一起结果会很小，会导致梯度消失问题，长短期记忆网络通过遗忘来减轻梯度消失的问题

# 反向传播算法
假设训练集只有一个实例$(x^{(1)},y^{(1)})$，神经网络是一个四层的神经网络，其中$K=4,S_L=4,L=4$：

前向传播算法： 
 
 <img src='https://i.loli.net/2018/12/02/5c03dbc252db6.png' width=500>
 
我们从最后一层的误差开始计算，误差是激活单元的预测$(a^{(4)}_k)$与实际值$(y^k)$之间的误差，$(k=1:K)$。     

1. 第四层：用δ来表示误差，则：$\delta^{(4)} = a^{(4)} -y$
 
2. 第三层：利用$\delta^{(4)}$来计算前一层的误差：$\delta^{(3)} = (\theta^{(3)})^T\delta^{(4)} * g^`(z^{(3)})$     
 其中$g^`(z^{(3)})$是 S 形函数的导数，$g^`(z^{(3)})=a^{(3)}*(1-a^{(3)})$。而 $(\theta^{(3)})^T\delta^{(4)} $则是权重导致的误差的和。
 
3. 继续计算第二层的误差：$\delta^{(2)} = (\theta^{(2)})^T\delta^{(3)} * g^`(z^{(2)})$ 
4. 第一层是输入变量，不存在误差。

有了所有的误差的表达式后，便可以计算代价函数的偏导数了，假设 $\lambda=0$，即我们不做任何正则化处理时有：
$$\frac{\partial}{\partial\theta^{(l)}_{ij}}j(\theta) = a^{(l)}_{j}\delta^{l+1}_i$$

**重要的是清楚地知道上面式子中上下标的含义：**

+ l 代表目前所计算的是第几层 
+ j 代表目前计算层中的激活单元的下标，也将是下一层的第 j 个输入变量的下标。 
+ i 代表下一层中误差单元的下标，是受到权重矩阵中第 i 行影响的下一层中的误差单元的下标。 

如果我们考虑正则化处理，并且我们的训练集是一个特征矩阵而非向量。在上面的特殊情况中，我们需要计算每一层的误差单元来计算代价函数的偏导数。在更为一般的情况中，我们同样需要计算每一层的误差单元，但是我们需要为整个训练集计算误差单元，此时的误差单元也是一个矩阵，我们用$\Delta^{(l)}_{ij}$来表示这个误差矩阵。第 l 层的第 i 个激活单元受到第 j 个参数影响而导致的误差。 

我们的算法表示为： 
 
  <img src='https://i.loli.net/2018/12/02/5c03df8488f60.png' width=500>
 
即首先用正向传播方法计算出每一层的激活单元，利用训练集的结果与神经网络预测的结果求出最后一层的误差，然后利用该误差运用反向传播法计算出直至第二层的所有误差。
 
在求出了$D^{(l)}_{ij}$之后，我们便可以计算代价函数的偏导数了，计算方法如下： 

 $$\Delta^{(l)}_{ij} := \frac{1}{m}\Delta^{(l)}_{ij}+\lambda\theta^{(l)}_{ij} if j\neq0$$
 $$\Delta^{(l)}_{ij} := \frac{1}{m}\Delta^{(l)}_{ij}+ if j=0$$